# YOLOv2 (YOLO9000:Better, Faster, Stronger)
- [paper](https://arxiv.org/pdf/1612.08242.pdf)

## Summary
- Batch Normalization
- Anchor Box

### Main Ideas

### **Better**

### Batch Normalization
- 논문에서는 모든 conv layer 뒤에 batch normalization을 추가하였습니다.
- 이를 통해 mAP 값이 2% 정도 향상되었습니다. 
- overfitting 없이 기타 regularization 방법이나 dropout을 제거할 수 있었습니다. 


### High Resolution Classifier
- YOLOv1은 Darknet을 224x224 크기로 pre-train시켰지만, detection task 시에는 448x448 크기의 이미지를 입력으로 사용합니다. 
- 이는 네트워크가 object detection task를 학습하면서 동시에 새로운 입력 이미지의 resolution(해상도)에 적응해야 함을 의미합니다. 
- YOLOv2는 처음부터 Darknet을 448x448 크기로 pre-train시켜 네트워크가 상대적으로 높은 해상도의 이미지에 적응할 시간을 제공합니다.
- 이러한 방법을 통해 mAP 값이 4% 정도 향상됩니다. 


### Convolutional with Anchor boxes
- YOLOv1은 각 grid cell의 bounding box의 좌표가 0~1 사이의 값을 가지도록 랜덤으로 설정한 뒤 학습을 통해 최적의 값을 찾아가는 과정을 거칩니다.
- 반면 Faster R-CNN 모델은 사전에 9개의 anchor box를 정의한 후 bounding box regression을 통해 x, y 좌표와 aspect ratio(offset)을 조정하는 과정을 거칩니다. 
- 좌표 대신 offset을 예측하는 문제가 보다 단순하고 네트워크가 학습하기 쉽다는 장점이 있습니다.

- YOLOv2는 anchor box를 도입하며, 이 과정에서 네트워크를 수정합니다. 
- 먼저 conv layer의 output이 보다 높은 resolution을 가지도록 pooling layer를 제거했습니다. 
- 또한 앞서 입력 이미지를 448x448 크기로 사용한다고 했지만, 네트워크를 줄여 416x416 크기의 입력 이미지를 사용합니다. 
- 입력 이미지를 수정한 이유는 최종 output feature map의 크기가 홀수가 되도록 하여, 
- feature map 내에 하나의 중심 cell(single center cell)이 존재할 수 있도록 하기 위함입니다. 
- 보통 객체의 크기가 큰 경우 이미지 내에서 중심을 차지하기 때문에, 하나의 중심 cell이 있으면 이를 잘 포착할 수 있기 때문입니다. 

416x416 크기의 입력 이미지를 네트워크에 입력할 경우 최종적으로 13x13 크기의 feature map을 얻을 수 있습니다(downsample ratio = 1/32). YOLO v1은 각 cell별로 2개의 bounding box를 예측하여 총 98(=7x7x2)개의 bounding box를 예측하지만, YOLO v2는 anchor box를 사용하여 보다 많은 수의 bounding box를 예측합니다. anchor box를 사용하지 않은 경우 mAP 값이 69.5%, recall값은 81%인 반면, anchor box를 사용한 경우 mAP 값은 69.2%, recall 값은 88%로 나왔습니다. anchor box를 사용하면 mAP 값이 감소하지만 recall 값이 상승하고, 이는 모델이 더 향상될 여지가 있음을 나타낸다고 합니다.

Object detection task에서 recall 값이 높다는 것은 모델이 실제 객체의 위치를 예측한 비율이 높음을 의미합니다. YOLO v1이 recall 값이 낮은 이유는 region proposal 기반의 모델에 비해 이미지 당 상대적으로 적은 수의 bounding box를 예측하기 때문입니다. 하지만 YOLO v2에서 anchor box를 통해 더 많은 수의 bounding box를 예측하면서 실제 객체의 위치를 보다 잘 포착하게 됩고, 이를 통해 recall 값이 상승하게 됩니다. 


### Dimension Clusters

- 기존에는 anchor box의 크기와 aspect ratio를 사전에 미리 정의(hand-picked)했습니다.
- 하지만 논문의 저자는 만약 네트워크가 더 좋은 조건으로 학습을 시작하기 위해 더 나은 사전 조건(prior)을 선택한다면 detection 성능이 향상될 것이라고 보았습니다. 
- 이를 위해 k-means clustering을 통해 최적의 prior를 탐색하는 방법을 제시합니다. 
- 이 때 데이터셋에 있는 모든 ground truth box의 width, height 값을 사용하여, k-means clustering 수행합니다. 
- $d(box,centroid) = 1 - IOU(box,centroid)$

일반적인 k-means clustering의 경우 유클리안 거리(Euclidean distance)를 통해 centroid와 sample간의 거리를 계산합니다. 하지만 이러한 distance metric을 사용할 경우 큰 bounding box는 작은 box에 비해 큰 error를 발생시키는 문제가 있습니다. 하지만 box의 크기와 무관하게 선택한 prior이 좋은 IoU 값을 가지도록 하기 위해서 논문의 저자는 위와 같은 새로운 distance metric을 사용합니다. 이는 box와 centroid의 IoU값이 클수록 겹치는 영역이 크기 때문에 거리가 가깝다는 점을 나타냅니다. 

